* Desarrollo de la estructura para responder a las consultas de la API.

En esta Notebook se prepararán los datos para tener listas las columnas referentes <br>
a los API endpoints.

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
df_reviews = pd.read_csv('data/user_reviews_clean.csv', encoding='utf-8')

In [3]:
df_games = pd.read_csv('data/steam_games_clean.csv', encoding='utf-8')

In [4]:
df_items = pd.read_csv('data/user_items_clean.csv', encoding='utf-8')

* Cantidad de Items y contenido Free <br>
En este apartado se van a realizar las columnas necesarias para luego presentar <br>
Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora

In [5]:
# Se hace un subconjunto con los que serán necesarios para la consulta
contenido_free_items = df_games[['price', 'release_year', 'developer', 'id']]
# Se renombra la columna para luego poder unirla con items
contenido_free_items = contenido_free_items.rename(columns={'id':'item_id'})


In [6]:
contenido_free_items.head(2)

,price,release_year,developer,item_id
0,4.99,2018.0,Kotoshiro,761140
1,4.99,2018.0,Kotoshiro,761140


Ahora se trabajará con el endpoint de userdata <br>
Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación <br>
en base a reviews.recommend y cantidad de items

In [7]:
# Se toman las columnas que serán necesarias
df_price = df_games[['price', 'id']]
# Se renombra la columna para luego poder unirla con items
df_price = df_price.rename(columns={'id':'item_id'})

In [8]:
df_price.head()

,price,item_id
0,4.99,761140
1,4.99,761140
2,4.99,761140
3,4.99,761140
4,4.99,761140


In [9]:
# Se toman las columnas que serán necesarias
df_price_items = df_items[['items_count', 'user_id', 'item_id']]
df_price_items.head(2)

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20


In [10]:
# Se hace un merge entre las dos tablas anteriores
df_price_items = df_price_items.merge(df_price, on='item_id',how='left')
df_price_items.head()

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99


In [11]:
# Se hace una revision de nulos para los precios 
df_price_items['price'].isna().sum()

1101130

In [12]:
# Los precios que tienen nulos, como no tienen asignado un precio se imputaran ceros 
df_price_items['price'] = df_price_items['price'].fillna(0)

In [13]:
df_price_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
10596256,7,76561198329548331,388490,0.00
10596257,7,76561198329548331,521570,0.00
10596258,7,76561198329548331,521570,0.00
10596259,7,76561198329548331,521570,0.00


In [14]:
# Ya que se termino con el merge, se puede eliminar la columna ites_id
df_price_items = df_price_items.drop('item_id', axis=1)

In [15]:
# Ahora sería de utilidad poder agrupar por usuario y saber cuanto es la sumna de price por usuario
df_price_user = df_price_items.groupby('user_id')['price'].sum().reset_index()
df_price_user

,user_id,price
0,--000--,1566.35
1,--ace--,348.58
2,--ionex--,169.86
3,-2SV-vuLB-Kg,942.80
4,-404PageNotFound-,3484.47
...,...,...
70907,zzonci,59.94
70908,zzoptimuszz,369.92
70909,zzydrax,209.88
70910,zzyfo,1382.17


En este punto se observa que disminuyeron las filas, porque ya tenemos una sola vez cada <br>
usuario, entonces para volver a tener en un mismo dataframe items_count pero con el mismo <br>
numero de filas, eliminamos duplicados del df de items y dejamos el primero únicamente

In [16]:
# Se retoman las columnas necesarias
df_items_count = df_price_items[['items_count','user_id']]
# Y se eliminan los duplicados para poder unirlos con el df_price_user
df_items_count = df_items_count.drop_duplicates(subset='user_id', keep='first')

In [17]:
df_items_count

,items_count,user_id
0,277,76561197970982479
496,888,js41637
2286,137,evcentric
2562,328,Riot-Punch
3096,541,doctr
...,...,...
10594763,321,76561198320136420
10595701,4,ArkPlays7
10595716,22,76561198323066619
10595742,177,76561198326700687


In [18]:
# Ya se tiene el mismo numero de filas, se procede a hacer el merge
df_user_data = df_items_count.merge(df_price_user, on='user_id', how='right')
df_user_data

,items_count,user_id,price
0,58,--000--,1566.35
1,44,--ace--,348.58
2,23,--ionex--,169.86
3,68,-2SV-vuLB-Kg,942.80
4,149,-404PageNotFound-,3484.47
...,...,...,...
70907,5,zzonci,59.94
70908,61,zzoptimuszz,369.92
70909,13,zzydrax,209.88
70910,84,zzyfo,1382.17


Ahora se trabajará con lo necesario para el endpoint UserForGenre que debe devolver <br>
el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación <br>
de horas jugadas por año de lanzamiento.

In [19]:
# Se seleccionan las que serán utiles para Horas por usuario
df_user_playtime = df_items[['playtime_forever', 'user_id', 'item_id']]
df_user_playtime.head()

,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50


In [20]:
# Se va a necesitar tambien items y genero
df_items_genre = df_games[['genres','id','release_year']]
df_items_genre = df_items_genre.rename(columns={'id':'item_id'})
df_items_genre

,genres,item_id,release_year
0,Action,761140,2018.0
1,Casual,761140,2018.0
2,Indie,761140,2018.0
3,Simulation,761140,2018.0
4,Strategy,761140,2018.0
...,...,...,...
55607,Indie,610660,2018.0
55608,Racing,610660,2018.0
55609,Simulation,610660,2018.0
55610,Casual,658870,2017.0


In [21]:
# Se hace un merge de los data frames anteriores
df_user_time_genre = df_user_playtime.merge(df_items_genre, on='item_id')
df_user_time_genre

,playtime_forever,user_id,item_id,genres,release_year
0,6,76561197970982479,10,Action,2000.0
1,0,js41637,10,Action,2000.0
2,0,Riot-Punch,10,Action,2000.0
3,93,doctr,10,Action,2000.0
4,108,corrupted_soul,10,Action,2000.0
...,...,...,...,...,...
9495126,164,76561198107283457,354280,Indie,2016.0
9495127,164,76561198107283457,354280,Simulation,2016.0
9495128,0,inven,433920,Adventure,2016.0
9495129,0,inven,433920,Indie,2016.0


In [22]:
# Se hace la agrupacion 
df_user_time_genre = df_user_time_genre.groupby(['genres', 'user_id', 'release_year'])['playtime_forever'].sum().reset_index()
# Se convierte a horas el tiempo jugado
df_user_time_genre['playtime_hrs'] = df_user_time_genre['playtime_forever']/60
# Se elimina la columna anterior de tiempo jugado
df_user_time_genre = df_user_time_genre.drop('playtime_forever', axis=1)
df_user_time_genre

,genres,user_id,release_year,playtime_hrs
0,Action,--000--,2009.0,88.816667
1,Action,--000--,2010.0,0.366667
2,Action,--000--,2011.0,108.700000
3,Action,--000--,2012.0,1822.433333
4,Action,--000--,2013.0,6.050000
...,...,...,...,...
3339195,Web Publishing,zachwgtv,2005.0,0.000000
3339196,Web Publishing,zachwgtv,2012.0,0.000000
3339197,Web Publishing,zepavil,2012.0,481.933333
3339198,Web Publishing,zepavil,2015.0,150.166667


Ahora se va a trabajar con Best Developer Year que devuelve el top 3 de desarrolladores <br>
con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [23]:
# Se cargan los datasets con las columnas que serán necesarias
df_dev = df_games[['developer','id','release_year']]
df_items_conect = df_items[['item_id','user_id']]
df_reviews_sentiment = df_reviews[['sentiment_analysis','reviews_recommend','user_id']]

In [24]:
# Se prepara para hacer el merge
df_dev = df_dev.rename(columns={'id':'item_id'})

In [25]:
# Se aplica el merge sobre items
df_dev_items = df_dev.merge(df_items_conect,on='item_id')

In [26]:
df_dev_items

,developer,item_id,release_year,user_id
0,Stainless Games Ltd,282010,1997.0,UTNerd24
1,Stainless Games Ltd,282010,1997.0,I_DID_911_JUST_SAYING
2,Stainless Games Ltd,282010,1997.0,76561197962104795
3,Stainless Games Ltd,282010,1997.0,r3ap3r78
4,Stainless Games Ltd,282010,1997.0,saint556
...,...,...,...,...
9495126,Valve,80,2004.0,76561198273508956
9495127,Valve,80,2004.0,76561198282090798
9495128,Valve,80,2004.0,943525
9495129,Valve,80,2004.0,76561198283312749


In [27]:
# se procede a unir con el ultimo dataframe
df_reviews_dev = df_reviews_sentiment.merge(df_dev_items,on='user_id')

In [28]:
df_reviews_dev.duplicated().sum()

8203476

In [29]:
df_reviews_dev.drop_duplicates(inplace=True)

In [30]:
df_reviews_dev.duplicated().sum()

0

In [31]:
df_reviews_dev

,sentiment_analysis,reviews_recommend,user_id,developer,item_id,release_year
0,2,True,76561197970982479,Valve,70,1998.0
1,2,True,76561197970982479,Arkane Studios,1700,2002.0
2,2,True,76561197970982479,"2K Boston,2K Australia",7670,2007.0
4,2,True,76561197970982479,"Epic Games, Inc.",13250,1998.0
5,2,True,76561197970982479,Oddworld Inhabitants,15710,1998.0
...,...,...,...,...,...,...
11371367,2,True,mijiperki,Secret Exit Ltd.,61600,2010.0
11371369,2,True,mijiperki,Bohemia Interactive,33930,2010.0
11371372,2,True,mijiperki,Ubisoft Montreal,33230,2010.0
11371374,2,True,mijiperki,Avalanche Studios,8190,2010.0


In [32]:
# Se elimina la columna que sirvio como union
df_reviews_dev = df_reviews_dev.drop(columns='item_id', axis=1)


In [33]:
df_reviews_dev.columns

Index(['sentiment_analysis', 'reviews_recommend', 'user_id', 'developer',
       'release_year'],
      dtype='object')

In [34]:
# Ahora solo dejamos los registros con review igual a true y con analisis de sentimiento = 2
df_best_developer_year = df_reviews_dev[(df_reviews_dev['reviews_recommend'] == True) & (df_reviews_dev['sentiment_analysis'] == 2)]

In [35]:
df_best_developer_year

,sentiment_analysis,reviews_recommend,user_id,developer,release_year
0,2,True,76561197970982479,Valve,1998.0
1,2,True,76561197970982479,Arkane Studios,2002.0
2,2,True,76561197970982479,"2K Boston,2K Australia",2007.0
4,2,True,76561197970982479,"Epic Games, Inc.",1998.0
5,2,True,76561197970982479,Oddworld Inhabitants,1998.0
...,...,...,...,...,...
11371367,2,True,mijiperki,Secret Exit Ltd.,2010.0
11371369,2,True,mijiperki,Bohemia Interactive,2010.0
11371372,2,True,mijiperki,Ubisoft Montreal,2010.0
11371374,2,True,mijiperki,Avalanche Studios,2010.0


In [36]:
df_best_developer_year.head()

,sentiment_analysis,reviews_recommend,user_id,developer,release_year
0,2,True,76561197970982479,Valve,1998.0
1,2,True,76561197970982479,Arkane Studios,2002.0
2,2,True,76561197970982479,"2K Boston,2K Australia",2007.0
4,2,True,76561197970982479,"Epic Games, Inc.",1998.0
5,2,True,76561197970982479,Oddworld Inhabitants,1998.0


In [37]:
df_best_developer_year = df_best_developer_year.copy()
df_best_developer_year['release_year'] = df_best_developer_year['release_year'].astype(int)

In [38]:
df_best_developer_year.head()

,sentiment_analysis,reviews_recommend,user_id,developer,release_year
0,2,True,76561197970982479,Valve,1998
1,2,True,76561197970982479,Arkane Studios,2002
2,2,True,76561197970982479,"2K Boston,2K Australia",2007
4,2,True,76561197970982479,"Epic Games, Inc.",1998
5,2,True,76561197970982479,Oddworld Inhabitants,1998


Finalmente, se retoma el data frame df_reviews_dev que nos puede servir para el endpoint <br>
de developer_reviews_analysis( desarrolladora : str ). Para el cuál será necesario: <br>
Desarrollador y analisis de sentimiento

In [39]:
# Se crea el data frame para developer_review_analysis llamado: df_dev_rev_analysis
df_dev_rev_analysis = df_reviews_dev[['developer','sentiment_analysis']]
df_dev_rev_analysis

,developer,sentiment_analysis
0,Valve,2
1,Arkane Studios,2
2,"2K Boston,2K Australia",2
4,"Epic Games, Inc.",2
5,Oddworld Inhabitants,2
...,...,...
11371367,Secret Exit Ltd.,2
11371369,Bohemia Interactive,2
11371372,Ubisoft Montreal,2
11371374,Avalanche Studios,2


In [40]:
df_dev_rev_analysis['sentiment_analysis'].nunique()

3

Por ultimo se sacan todos los datasets para poder tenerlos disponibles para la API

In [41]:
# Parquet - contenido_free_ruta
contenido_free_ruta = 'data/contenido_free_items.parquet'
contenido_free_items.to_parquet(contenido_free_ruta, index=False)

In [42]:
# Parquet - df_user_data
user_data_ruta = 'data/user_data_price.parquet'
df_user_data.to_parquet(user_data_ruta, index=False)

In [43]:
# Parquet - UserForGenre
user_time_genre_ruta = 'data/user_for_genre.parquet'
df_user_time_genre.to_parquet(user_time_genre_ruta, index=False)

In [44]:
# Parquet - best_developer_year
developer_year_ruta = 'data/best_developer_year.parquet'
df_best_developer_year.to_parquet(developer_year_ruta, index=False)


In [45]:
# Parquet - developer_review_analysis
dev_rev_analysis_ruta = 'data/dev_rev_analysis.parquet'
df_dev_rev_analysis.to_parquet(dev_rev_analysis_ruta, index=False)

In [46]:
user_reviews_ruta = 'data/user_reviews_clean.parquet'
df_reviews.to_parquet(user_reviews_ruta, index=False)